<span style="color: green; font-size: 40px; font-weight: bold;">Lab 5 (Clusterização (Agrupamento)) </span>

<br> <br>

# Agrupando Veículos Por Similaridade

<br>

### Contexto

Neste projeto, abordaremos uma aplicação prática de técnicas de aprendizado não supervisionado, especificamente a **clusterização de veículos** com base em suas características. Diferente de problemas supervisionados, onde as respostas corretas são conhecidas e usadas para treinar o modelo, aqui o objetivo é descobrir grupos naturais ou clusters nos dados. Através deste processo, pretendemos identificar padrões e agrupamentos de veículos que compartilham características similares, como tipo de combustível, número de portas, tipo de direção, potência, consumo de combustível, entre outros. Esta técnica é útil em diversos contextos, como segmentação de mercado, identificação de perfis de clientes, e análise de produtos.

<br>

### Objetivo

O objetivo deste projeto é **agrupar veículos em clusters com base em suas características similares**. Utilizando um algoritmo de aprendizado não supervisionado, como o K-means, o modelo analisará os dados de entrada e identificará grupos de veículos que compartilham características comuns, sem a necessidade de rótulos predefinidos.

<br>

### Pergunta de Negócio Principal

> "Como podemos agrupar veículos de acordo com suas características para identificar grupos de similaridade?"

<br>

### Entregável

O entregável deste projeto será uma **análise de agrupamento (clusterização) de veículos**, onde veículos similares serão agrupados em clusters. Esses clusters podem ser usados para diversas finalidades, como identificar segmentos de mercado, definir estratégias de marketing ou até mesmo analisar padrões de fabricação e consumo. O projeto incluirá a preparação dos dados, a escolha do algoritmo de clusterização, a execução do modelo e a interpretação dos resultados.

<br>

### Sobre o Conjunto de Dados

Os dados utilizados neste projeto contêm várias características dos veículos, incluindo fabricante, tipo de combustível, número de portas, tipo de direção, número de cilindros, potência, consumo de combustível na cidade e na estrada, e preço. Estes atributos servirão como base para identificar similaridades e agrupar os veículos.

<br>

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>fabricante</td>
    <td>string</td>
    <td>Nome do fabricante do veículo.</td>
  </tr>
  <tr>
    <td>tipo_combustivel</td>
    <td>string</td>
    <td>Tipo de combustível utilizado pelo veículo (gasolina, diesel, etc.).</td>
  </tr>
  <tr>
    <td>aspirado</td>
    <td>string</td>
    <td>Indicação se o motor é aspirado ou não.</td>
  </tr>
  <tr>
    <td>portas</td>
    <td>integer</td>
    <td>Número de portas do veículo.</td>
  </tr>
  <tr>
    <td>tipo</td>
    <td>string</td>
    <td>Tipo de carroceria do veículo (hatchback, sedan, etc.).</td>
  </tr>
  <tr>
    <td>direcao</td>
    <td>string</td>
    <td>Tipo de direção do veículo (traseira, dianteira, etc.).</td>
  </tr>
  <tr>
    <td>cilindros</td>
    <td>integer</td>
    <td>Número de cilindros do motor.</td>
  </tr>
  <tr>
    <td>horsepower</td>
    <td>integer</td>
    <td>Potência do motor, medida em cavalos de força.</td>
  </tr>
  <tr>
    <td>rpm</td>
    <td>integer</td>
    <td>Rotações por minuto (RPM) em que a potência máxima é alcançada.</td>
  </tr>
  <tr>
    <td>consumo_cidade</td>
    <td>integer</td>
    <td>Consumo de combustível na cidade (milhas por galão).</td>
  </tr>
  <tr>
    <td>consumo_estrada</td>
    <td>integer</td>
    <td>Consumo de combustível na estrada (milhas por galão).</td>
  </tr>
  <tr>
    <td>preco</td>
    <td>integer</td>
    <td>Preço do veículo em dólares.</td>
  </tr>
</table>

<br><br><br>

# Importando Pacotes

In [1]:
# Importa o findspark e inicializa
import findspark
findspark.init()

# Imports
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
import matplotlib.pylab as plt
%matplotlib inline

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

<br> <br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Preparando o Ambiente Spark</span>

In [2]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

# Se houver uma sessão Spark ativa, encerre-a
if 'sc' in globals():
    sc.stop()

if 'spark' in globals():
    spark.stop()


# Criando o Spark Context
conf = SparkConf().setAppName("Lab5") \
                  .set("spark.ui.showConsoleProgress", "false") \
                  .set("spark.executor.heartbeatInterval", "20s") \
                  .set("spark.eventLog.enabled", "false") \
                  .set("spark.sql.shuffle.partitions", "2") \
                  .set("spark.sql.debug.maxToStringFields", "100") \
                  .set("spark.executor.memory", "4g") \
                  .set("spark.driver.memory", "4g") \
                  .set("spark.driver.maxResultSize", "2g")  # Configuração adicional para limitar o tamanho do resultado

# Criar o Spark Context e a Spark Session
sc = SparkContext(conf=conf)
spSession = SparkSession.builder.config(conf=conf).getOrCreate()

# Ajustar o nível de log para ERROR
sc.setLogLevel("ERROR")

# Configurar log4j para suprimir avisos (deixar como comentário e volta ao normal)
log4j_logger = sc._jvm.org.apache.log4j
log4j_logger.LogManager.getLogger("org").setLevel(log4j_logger.Level.ERROR)
log4j_logger.LogManager.getLogger("akka").setLevel(log4j_logger.Level.ERROR)

# Visualizar o objeto spark_session
spSession

24/08/12 12:29:33 WARN Utils: Your hostname, eduardo-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface wlp0s20f3)
24/08/12 12:29:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/12 12:29:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/12 12:29:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Carregando os Dados</span>

- Os dados serão carregados a partir de um arquivo CSV e gerados como um RDD (Resilient Distributed Dataset) no Apache Spark. O RDD é uma estrutura de dados distribuída que permite o processamento paralelo em um cluster, otimizando a performance.

In [3]:
# Carregando os dados e gerando um RDD
carrosRDD = sc.textFile("Lab/dados/dataset5.csv")

# Tipo
print(type(carrosRDD), '\n')

# Colocando o RDD em cache. Esse processo otimiza a performance
print(carrosRDD.cache(), '\n')

# Número de registros
print(carrosRDD.count(), '\n')

<class 'pyspark.rdd.RDD'> 

Lab/dados/dataset5.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 

198 



In [4]:
# Visualizando as primeiras linhas
print(carrosRDD.take(5))

['fabricante,tipo_combustivel,aspirado,portas,tipo,direcao,cilindros,horsepower,rpm,consumo_cidade,consumo_estrada,preco', 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118', 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151', 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195', 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348']


In [5]:
## Visualizando primeiras 4 linhas com Pandas (Apenas para visualização)

import pandas as pd

# Obter as primeiras 5 linhas do RDD
linhas = carrosRDD.take(5)

# Dividir as linhas em colunas utilizando o ponto-e-vírgula como delimitador
colunas = linhas[0].split(";")
dados_formatados = [linha.split(";") for linha in linhas[1:]]

# Criar um DataFrame Pandas com as colunas e os dados formatados
df = pd.DataFrame(dados_formatados, columns=colunas)

# Mostrar o DataFrame
display(df)

,"fabricante,tipo_combustivel,aspirado,portas,tipo,direcao,cilindros,horsepower,rpm,consumo_cidade,consumo_estrada,preco"
0,"subaru,gas,std,two,hatchback,fwd,four,69,4900,..."
1,"chevrolet,gas,std,two,hatchback,fwd,three,48,5..."
2,"mazda,gas,std,two,hatchback,fwd,four,68,5000,3..."
3,"toyota,gas,std,two,hatchback,fwd,four,62,4800,..."


<br><br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;"> Análise Exploratória Inicial dos Dados </span>

<br>

### Criação de Função Para Análise Inicial

In [6]:
import pandas as pd
import re

def funcao_analise_inicial(df):
    # Configurar Pandas para exibir todas as linhas
    pd.set_option('display.max_rows', None)

    # Informações do DataFrame
    print('\n\n INFO \n\n')
    df.info()
    print('\n\n ------------------------------------------------------------------------------------------ \n\n')

    # Verifica se há valores ausentes e duplicados
    valores_ausentes = df.isna().sum().sum() > 0
    valores_duplicados = df.duplicated().sum() > 0

    # Nomes das variáveis com valores ausentes
    variaveis_ausentes = df.columns[df.isna().any()].tolist()

    # Número de linhas duplicadas
    num_linhas_duplicadas = df.duplicated().sum()

    # Porcentagem de linhas duplicadas
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / len(df)) * 100

    # Exibe o resultado sobre valores ausentes e duplicados
    print("\n\nExistem valores ausentes:", valores_ausentes)
    if valores_ausentes:
        print("\nVariáveis com valores ausentes:", variaveis_ausentes)
    else:
        print("\nNenhuma variável possui valores ausentes.")

    print("\n\nExistem valores duplicados:", valores_duplicados)
    if valores_duplicados:
        print("\nNúmero de Linhas Duplicadas:", num_linhas_duplicadas)
        print("\nPorcentagem de Linhas Duplicadas: {:.2f}%".format(porcentagem_linhas_duplicadas))
    else:
        print("\nNenhuma variável possui valores duplicados.")
    
    # Verificação de caracteres especiais
    caracteres_especiais = re.compile('[@_!#$%^&*<>()?/\\|}{~:]')   # nenhum caracter removido
    colunas_com_caracteres_especiais = {}

    for coluna in df.columns:
        if df[coluna].dtype == 'object':  # Verifica apenas colunas de texto
            contem_caracteres_especiais = df[coluna].apply(lambda x: bool(caracteres_especiais.search(x) if isinstance(x, str) else False)).any()
            if contem_caracteres_especiais:
                indices_com_caracteres_especiais = df[coluna][df[coluna].apply(lambda x: bool(caracteres_especiais.search(x) if isinstance(x, str) else False))].index.tolist()
                colunas_com_caracteres_especiais[coluna] = indices_com_caracteres_especiais

    # Exibe o resultado sobre caracteres especiais
    print("\n\nExistem caracteres especiais nas colunas:", bool(colunas_com_caracteres_especiais))
    if colunas_com_caracteres_especiais:
        print("\nColunas com caracteres especiais e os índices:")
        for coluna, indices in colunas_com_caracteres_especiais.items():
            print(f"\n Coluna [ {coluna} ]: Índices com caracteres especiais {indices}")
    else:
        print("\nNenhuma coluna possui caracteres especiais.")

print('A função foi criada com sucesso.')

A função foi criada com sucesso.


<br>

### Transformando dados carregados em RDD para dataframe do Pandas (apenas para Análise Inicial)

- Vamos realizar análise exploratória através da função acima. RDDs são ótimos para processamento, mas ruins para exploração, então converteremos o RDD para DataFrame Spark e então para DataFrame Pandas (**não é possível converter diretamente objeto RDD para objeto Pandas**).

In [7]:
# Converte RDD para DataFrame Spark, corrigindo a separação de colunas
header = carrosRDD.first().replace('"', '').split(';')
df_spark_com_cabecalho = carrosRDD.map(lambda x: x.replace('"', '').split(';')).toDF(header)

# Remover a primeira linha do DataFrame, que é o cabeçalho
df_spark_com_cabecalho = df_spark_com_cabecalho.filter(df_spark_com_cabecalho[header[0]] != header[0])

# Verificar o tipo do objeto
print(type(df_spark_com_cabecalho), '\n')

# Converte DataFrame Spark para DataFrame Pandas
df_pandas = df_spark_com_cabecalho.toPandas()

# Visualizando as primeiras linhas do DataFrame Pandas
display(df_pandas.head())

<class 'pyspark.sql.dataframe.DataFrame'> 



,"fabricante,tipo_combustivel,aspirado,portas,tipo,direcao,cilindros,horsepower,rpm,consumo_cidade,consumo_estrada,preco"
0,"subaru,gas,std,two,hatchback,fwd,four,69,4900,..."
1,"chevrolet,gas,std,two,hatchback,fwd,three,48,5..."
2,"mazda,gas,std,two,hatchback,fwd,four,68,5000,3..."
3,"toyota,gas,std,two,hatchback,fwd,four,62,4800,..."
4,"mitsubishi,gas,std,two,hatchback,fwd,four,68,5..."


<br>

### Visualizando Função para Análise Inicial

In [8]:
funcao_analise_inicial(df_pandas)



 INFO 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 1 columns):
 #   Column                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                  --------------  ----- 
 0   fabricante,tipo_combustivel,aspirado,portas,tipo,direcao,cilindros,horsepower,rpm,consumo_cidade,consumo_estrada,preco  197 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


 ------------------------------------------------------------------------------------------ 




Existem valores ausentes: False

Nenhuma variável possui valores ausentes.


Existem valores duplicados: False

Nenhuma variável possui valores duplicados.


Existem caracteres especiais nas colunas: False

Nenhuma coluna possui caracteres especiais.


### Resumo

- Não será necessário nenhum tratamento de **Limpeza de Dados**.

<br> <br> <br>